In [31]:
# this code section loads all needed data to a local folder and immediately creates a backup
import os, urllib.request, json
import pandas as pd
import shutil
from datetime import datetime
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile


# get newest available string to download FOPH data
zipurl = ''
with urllib.request.urlopen("https://www.covid19.admin.ch/api/data/context") as url:
    data = json.loads(url.read().decode())
    zipurl = data["sources"]["zip"]["csv"]

# download the FOPH data
with urlopen(zipurl) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        zfile.extractall('data/FOPH')

# download the Google Mobility data
zipurl = 'https://www.gstatic.com/covid19/mobility/Region_Mobility_Report_CSVs.zip'
with urlopen(zipurl) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        zfile.extractall('data/GoogleMobility')

# download the Intervista Mobility data
zipurl = 'https://www.intervista.ch/media/2020/03/Download_Mobilit%C3%A4ts-Monitoring_Covid-19.zip'
with urlopen(zipurl) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        zfile.extractall('data/IntervistaMobility')

# KOF strigency index data
df = pd.read_csv('https://datenservice.kof.ethz.ch/api/v1/public/sets/stringency_plus_web?mime=csv&df=Y-m-d')
if not os.path.exists('data/KOF'):
    os.makedirs('data/KOF')
df.to_csv('data/KOF/KOFStrigencyIndex.csv')
      
# Oxford COVID-19 Government Response Tracker
df = pd.read_csv('https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv')
if not os.path.exists('data/Oxford'):
    os.makedirs('data/Oxford')
df.to_csv('data/Oxford/OxfordStrigencyIndex.csv')

# TODO: weather (open weather api key requested, but i will only get past data maximum one year back)

# TODO: nextstrain: https://nextstrain.org/groups/swiss/ncov/switzerland
# I failed to automatically download the data at the moment because the link has a automatically generated id like 
# https://nextstrain.org/656098e0-bf62-d941-bf5f-e63262053c4a which is only valid once and the fetch method described
# in the documentation did not work (maybe it only works for certain data sets)

# this is manual work
# TODO: holidays: https://de.wikipedia.org/wiki/Feiertage_in_der_Schweiz
# TODO: school vacations: https://www.schulferien.org/schweiz/ferien/
# TODO: measures.csv cantonal/federal

# International data for bordering countries (only weekly because germany, also earliest data point is from week 13 2020)
df = pd.read_csv('https://opendata.ecdc.europa.eu/covid19/subnationalcaseweekly/csv')
if not os.path.exists('data/ECDC'):
    os.makedirs('data/ECDC')
df.to_csv('data/ECDC/ECDCsubnationalcaseweekly.csv')

# International data for bordering countries (only country level data)
df = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv')
if not os.path.exists('data/OWID'):
    os.makedirs('data/OWID')
df.to_csv('data/OWID/OWIDcoviddata.csv')

# create a backup of the data we just loaded
if not os.path.exists('backups'):
    os.makedirs('backups')
now = datetime.now()
backupname = now.strftime("backup-%Y-%m-%d-%H-%M-%S")
shutil.copytree('data', 'backups/'+backupname)

'backups/backup-2021-03-03-22-19-06'